# Skelettschätzer

In [ ]:
!pip install ipywidgets tomli ray
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

import os
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output, HTML
import ray
import subprocess
import yaml
import uuid

ℹ Bitte aktivieren Sie die Logs, denn einige Ausgaben können nur darüber angezeigt werden.
`View > Show Log Console`

## 1. Auswählen von Videos
Wählen Sie zunächst Videos aus, die verarbeitet werden. Sie können mp4-Dateien ins Arbeitsverzeichnis laden. Außerdem können Sie YouTube-Videos per Link einfügen, die dann verarbeitet werden.

In [ ]:
# Data-Ordner finden
with open("./.env", "r") as f:
    env = [line.replace("\n", "").split("=") for line in f.readlines()]
env = {e[0]: e[1] for e in env}
data_folder = env["INPUT_FOLDER"]
input_folder = os.path.join(data_folder, "input")
input_folder_parsed = Path(input_folder)
# YouTube-Videos suchen
if "input.txt" in os.listdir(input_folder):
    with open(os.path.join(input_folder, "input.txt"), "r") as f:
        yt_links = f.read()
else:
    yt_links = ""

# ---------- Widgets ----------

# Datei-Upload (nur mp4 zulassen)
upload_widget = widgets.FileUpload(
  accept='.mp4',
  multiple=True,
  description='MP4 hochladen'
)

# Mehrzeiliges Eingabefeld für URLs
url_textarea = widgets.Textarea(
  placeholder='Gib hier YouTube-Links ein (einer pro Zeile)',
  description='URLs:',
  layout=widgets.Layout(width='100%', height='100px'),
  value=yt_links
)

# Button zum Löschen aller MP4-Dateien
delete_button = widgets.Button(
  description='Alle MP4-Dateien löschen',
  button_style='danger'
)

# Bestätigungsbutton
confirm_button = widgets.Button(
  description='Auswahl bestätigen',
  button_style='success'
)

# Ausgabe für die Dateiliste
file_list_output = widgets.Output()

# ---------- Funktionen ----------

def save_uploaded_files(change):
    for fileinfo in upload_widget.value:
        file_path = input_folder_parsed / fileinfo["name"]
        with open(file_path, 'wb') as f:
            f.write(fileinfo['content'])
    upload_widget.value = []
    update_file_list()

def update_file_list():
  with file_list_output:
    clear_output()
    mp4_files = list(input_folder_parsed.glob('*.mp4'))
    if mp4_files:
      display(HTML('<b>Vorhandene MP4-Dateien:</b>'))
      for file in mp4_files:
        display(HTML(f'📁 {file.name}'))
    else:
      display(HTML('<i>Keine MP4-Dateien im Ordner.</i>'))

def delete_all_mp4_files(b):
  for file in input_folder_parsed.glob('*.mp4'):
    file.unlink()
  update_file_list()

def confirm_selection(b):
  urls = url_textarea.value.strip().splitlines()
  print('✅ Bestätigt:')
  print(f'- {len(list(input_folder_parsed.glob("*.mp4")))} MP4-Dateien im Ordner')
  print(f'- {len(urls)} URLs eingegeben')
  for url in urls:
    print(f'  • {url.strip()}')
  with open(input_folder_parsed / 'input.txt', 'w') as f:
      f.write("\n".join(urls))

# ---------- Event-Handler verbinden ----------

upload_widget.observe(save_uploaded_files, names='value')
delete_button.on_click(delete_all_mp4_files)
confirm_button.on_click(confirm_selection)

# ---------- UI anzeigen ----------

display(widgets.VBox([
  upload_widget,
  url_textarea,
  delete_button,
  confirm_button,
  file_list_output
]))

# Initiale Anzeige
update_file_list()

## 2. Ausführen der Skelettschätzung
Die Skelettschätzungen laufen auf Docker-Containern. Mit einem Klick auf den Button starten Sie die Ausführung.

⚠ Das kann eine Weile dauern.

In [ ]:
def get_num_gpus():
    try:
        result = subprocess.run(["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"],
                                capture_output=True,
                                text=True,
                                check=True)
        return len(result.stdout.strip().split("\n"))
    except:
        print("WARNUNG: Anzahl der GPUs konnte nicht ermittelt werden.")
        return 1

ray.init(num_gpus=get_num_gpus())

def run_container(service):
    print(f"⏳ Docker-Container läuft: {service}")
    container_name = f"{service}-{uuid.uuid4().hex[:8]}"
    result = subprocess.run(["docker", "compose", "run", "--no-deps", "--name", container_name, service])
    if result.returncode == 0:
        print(f"✔ Docker-Container beendet: {service}")
    else:
        print(f"❌ Fehler beim Ausführen von: {service} (Exit-Code: {result.returncode})")
        print(f"📜 Container-Logs von {container_name}:")
        subprocess.run(["docker", "logs", container_name])

@ray.remote(num_gpus=1)
def run_container_as_service(poser):
    run_container(poser)

def run(posers=None):
    if posers is None:
        posers = get_services()
    subprocess.run(["docker", "compose", "build"])
    run_container("input-loader")
    tasks = [run_container_as_service.remote(poser) for poser in posers]
    ray.get(tasks)
    run_container("codecs")
    run_container("data-transformer")

def get_services(docker_compose_path="./docker-compose.yml"):
    with open(docker_compose_path) as f:
        dc = yaml.load(f, Loader=yaml.FullLoader)
    services = list(dc["services"].keys())
    services.remove("input-loader")
    services.remove("data-transformer")
    services.remove("codecs")
    return services

# Widgets
status_output = widgets.Output()

def confirm(b):
    with status_output:
        clear_output()
        display(HTML('<p style="color:orange;font-weight:bold;">⏳ Programm wird ausgeführt ...</p>'))
    run()
    with status_output:
        clear_output(wait=True)
        display(HTML('<p style="color:darkgreen;font-weight:bold;">✅ Ausführung beendet</p>'))

# Button
confirm_button = widgets.Button(
  description='Programm starten',
  button_style='success'
)
confirm_button.on_click(confirm)

# Anzeige
display(widgets.VBox([confirm_button, status_output]))

Statusmeldungen während der Ausführung der Container können Sie sich in den Logs ansehen:
`View > Show Log Console`